In [1]:
import pandas as pd
import numpy as np
import json
import re

In [2]:
with open("2018-02-14_focus_discussion/2018-02-14_focus_group.json", 'r', encoding='UTF-8') as focus_group:
    focus = json.load(focus_group)

In [3]:
with open("krx_code.json", 'r', encoding='UTF-8') as krx:
    krx_code = json.load(krx)

In [4]:
with open("2018-02-14_opening_increase.json", 'r', encoding='UTF-8') as oi:
    INCREASE = json.load(oi)

In [5]:
with open("2018-02-14_price_and_everything.json", 'r', encoding='UTF-8') as oi:
    price = json.load(oi)

In [54]:
time = []

prefix = "2018-02-14_focus_discussion/discussion_2018-02-14"
for hour in range(9,16):
    for minute in range(0, 6):
        time.append(prefix + " " + (("0" + str(hour)) if hour <= 9 else str(hour))+ \
                    ":" + str(minute) + "0.json")

In [55]:
del time[41]
del time[40]

In [56]:
discussion_list = []
for x in time[1: ]:
    with open(x, 'r', encoding='UTF-8') as f:
        discussion = json.load(f)        
        discuss_df = pd.DataFrame(discussion, columns = ["post_num", \
                            "unique_id", "click", "like", "dislike", 
                            "name", "time"])
        reset_col = ["name", "time", "post_num", "unique_id", "click", \
                     "like", "dislike"]
        discuss_df = discuss_df[reset_col]
        discussion_list.append(discuss_df)        

In [57]:
def list_to_df(df_list, column_names, key_list):
    total = pd.DataFrame(columns=column_names)
    for ind, df in enumerate(df_list):
        if ((ind + 1) % 3  == 0) and (ind != 2):
            df_total = df.merge(df_list[ind - 5], on = \
                                key_list).merge(df_list[ind - 4], \
                                on = key_list).merge(df_list[ind - 3], on = key_list)
            df_total.columns = column_names
            total = pd.concat([total, df_total], axis = 0)
            total = total[total["name"].isin(INCREASE)]
    return total    

In [58]:
COLUMN_NAMES=["name", "time", "post_num", "unique_id", "click", \
              "like", "dislike", "time_1", "post_num_1", "unique_id_1", "click_1", \
              "like_1", "dislike_1", "time_2", "post_num_2", "unique_id_2", "click_2", \
              "like_2", "dislike_2", "time_3", "post_num_3", "unique_id_3", "click_3", \
              "like_3", "dislike_3"]
discuss_df = list_to_df(discussion_list, COLUMN_NAMES, 
                        ['name']).reset_index().drop(["index"], axis = 1)

In [59]:
price_df = pd.DataFrame(price, columns = ["index", "code", "name", "time", "price", \
                                         "price_dif", "sell", "buy", "volume", "variation"])
price_df = price_df[["code", "name", "time", "price", \
                    "price_dif", "sell", "buy", "volume", "variation"]][1:]
price_df = price_df[price_df["time"] != "2018-02-14 09:00"]

In [60]:
time_list = price_df['time'].unique().tolist()
price_df_list = []
for time in time_list:
    df = price_df[price_df["time"] == time]
    df = df[["code", "name", "time", "price", \
            "price_dif", "sell", "buy", "volume", "variation"]]
    price_df_list.append(df)

In [61]:
COLUMN_PRICE=["code", "name", "time", "price", "price_dif", "sell", 
              "buy", "volume", "variation", "time_1", "price_1", 
              "price_dif_1", "sell_1", "buy_1", "volume_1", "variation_1",
              "time_2", "price_2", "price_dif_2", "sell_2", "buy_2", 
              "volume_2", "variation_2", "time_3", "price_3", 
              "price_dif_3", "sell_3", "buy_3", "volume_3", "variation_3"]
price_df = list_to_df(price_df_list, COLUMN_PRICE, \
                      ['code', 'name']).reset_index().drop(["index"], axis = 1)

In [62]:
price_df.set_index("name")
discuss_df.set_index("name")

,time,post_num,unique_id,click,like,dislike,time_1,post_num_1,unique_id_1,click_1,...,unique_id_2,click_2,like_2,dislike_2,time_3,post_num_3,unique_id_3,click_3,like_3,dislike_3
name,,,,,,,,,,,,,,,,,,,,,
호텔신라,2018-02-14 10:00,7,5,937,13,6,2018-02-14 09:10,4,3,260,...,4,459,7,4,2018-02-14 09:30,5,4,571,7,4
대한광통신,2018-02-14 10:00,6,5,627,6,4,2018-02-14 09:10,0,0,0,...,0,0,0,0,2018-02-14 09:30,3,3,142,1,3
포스코 ICT,2018-02-14 10:00,3,3,1123,2,9,2018-02-14 09:10,2,2,542,...,2,656,1,4,2018-02-14 09:30,3,3,781,1,5
한화케미칼,2018-02-14 10:00,9,8,1094,10,15,2018-02-14 09:10,2,2,405,...,3,529,2,7,2018-02-14 09:30,6,6,706,3,8
지엘팜텍,2018-02-14 10:00,4,4,1416,9,14,2018-02-14 09:10,3,3,472,...,4,751,4,10,2018-02-14 09:30,4,4,970,8,12
코웰패션,2018-02-14 10:00,2,2,267,1,0,2018-02-14 09:10,1,1,67,...,1,91,1,0,2018-02-14 09:30,1,1,121,1,0
대한항공,2018-02-14 10:00,5,3,469,1,4,2018-02-14 09:10,0,0,0,...,1,19,0,0,2018-02-14 09:30,2,2,70,0,0
대한뉴팜,2018-02-14 10:00,1,1,63,0,3,2018-02-14 09:10,1,1,5,...,1,36,3,3,2018-02-14 09:30,1,1,64,3,3
레드캡투어,2018-02-14 10:00,0,0,0,0,0,2018-02-14 09:10,0,0,0,...,0,0,0,0,2018-02-14 09:30,0,0,0,0,0


In [64]:
total = pd.merge(price_df, discuss_df, on = ['name', 'time', 'time_1', 'time_2', 'time_3'])
total = total[['name', 'code', 'time', 'price', 'time_1', "price_1", 
              "price_dif_1", "sell_1", "buy_1", "volume_1", "variation_1", 'post_num_1',
              'unique_id_1', 'click_1', 'like_1', 'dislike_1', 
              'time_2', 'price_2', "price_dif_2", "sell_2", "buy_2", 
              "volume_2", "variation_2", 'post_num_2', 'unique_id_2', 'click_2', 'like_2',
              'dislike_2', 'time_3', 'price_3', "price_dif_3", "sell_3", "buy_3", 
              "volume_3", "variation_3", 'post_num_3', 'unique_id_3', 'click_3', 'like_3', 'dislike_3']]

In [51]:
total.columns

Index(['name', 'code', 'time', 'price', 'time_1', 'price_1', 'price_dif_1',
       'sell_1', 'buy_1', 'volume_1', 'variation_1', 'unique_id_1', 'click_1',
       'like_1', 'dislike_1', 'time_2', 'price_2', 'price_2', 'price_dif_2',
       'sell_2', 'buy_2', 'volume_2', 'variation_2', 'post_num_2',
       'unique_id_2', 'click_2', 'like_2', 'dislike_2', 'time_3', 'price_3',
       'price_dif_3', 'sell_3', 'buy_3', 'volume_3', 'variation_3',
       'unique_id_3', 'click_3', 'like_3', 'dislike_3', 'mkt_cap', 'kospi',
       'kosdaq', 'trash'],
      dtype='object')

In [65]:
import numpy as np
import re

with open("company_size.json", 'r', encoding='UTF-8') as f:
    company_size = json.load(f)

company_df = pd.DataFrame({"company_name": np.zeros(0), \
                           "code": np.zeros(0), "market": np.zeros(0), 
                          "company_size": np.zeros(0)})

company_df = pd.DataFrame(company_size, columns = ["name", "code", "market", "size"])

KOSPI = []
KOSDAQ = []
TRASH = []
MKT_CAP = {}

for idx, row in company_df.iterrows():
    if re.sub('[0-9 ,위]', '', row["market"]) == "코스피":
        KOSPI.append(row["code"])
    elif re.sub('[0-9 ,위]', '', row["market"]) == "코스닥":
        KOSDAQ.append(row["code"])
    else:
        TRASH.append(row["code"])
    
    MKT_CAP[row["code"]] = int(re.sub('[조억원,]', '', row["size"]))

In [66]:
total["mkt_cap"] = np.nan
total["kospi"] = np.nan
total["kosdaq"] = np.nan
total["trash"] = np.nan

In [67]:
total

,name,code,time,price,time_1,price_1,price_dif_1,sell_1,buy_1,volume_1,...,variation_3,post_num_3,unique_id_3,click_3,like_3,dislike_3,mkt_cap,kospi,kosdaq,trash
0,호텔신라,008770,2018-02-14 10:00,"83,400",2018-02-14 09:10,"84,700","1,800","84,800","84,700","29,336",...,"2,036",5,4,571,7,4,NaN,NaN,NaN,NaN
1,대한광통신,010170,2018-02-14 10:00,"7,050",2018-02-14 09:10,"7,060",90,"7,060","7,050","101,713",...,"3,467",3,3,142,1,3,NaN,NaN,NaN,NaN
2,포스코 ICT,022100,2018-02-14 10:00,"8,320",2018-02-14 09:10,"8,310",0,"8,330","8,310","44,311",...,"1,712",3,3,781,1,5,NaN,NaN,NaN,NaN
3,한화케미칼,009830,2018-02-14 10:00,"32,350",2018-02-14 09:10,"32,700",550,"32,700","32,650","32,647",...,"3,102",6,6,706,3,8,NaN,NaN,NaN,NaN
4,지엘팜텍,204840,2018-02-14 10:00,"4,760",2018-02-14 09:10,"4,680",50,"4,700","4,680","114,565",...,"2,844",4,4,970,8,12,NaN,NaN,NaN,NaN
5,코웰패션,033290,2018-02-14 10:00,"5,350",2018-02-14 09:10,"5,270",90,"5,280","5,270","22,780",...,"1,530",1,1,121,1,0,NaN,NaN,NaN,NaN
6,대한항공,003490,2018-02-14 10:00,"35,900",2018-02-14 09:10,"35,700",300,"35,750","35,700","7,560",...,"1,028",2,2,70,0,0,NaN,NaN,NaN,NaN
7,대한뉴팜,054670,2018-02-14 10:00,"13,750",2018-02-14 09:10,"13,800",0,"13,800","13,750","2,556",...,365,1,1,64,3,3,NaN,NaN,NaN,NaN
8,레드캡투어,038390,2018-02-14 10:00,"15,350",2018-02-14 09:10,"15,350",100,"15,350","15,200",7,...,1,0,0,0,0,0,NaN,NaN,NaN,NaN
9,이마트,139480,2018-02-14 10:00,"290,000",2018-02-14 09:10,"283,500","5,000","284,000","283,500","8,469",...,75,1,1,23,2,0,NaN,NaN,NaN,NaN


In [68]:
for index, row in total.iterrows():
    mkt_cap = MKT_CAP[row["code"]]
    if row["code"] in KOSPI:
        kospi = 1
    else:
        kospi = 0
    
    if row["code"] in KOSDAQ:
        kosdaq = 1
    else:
        kosdaq = 0
        
    if row["code"] in TRASH:
        trash = 1
    else:
        trash = 0
    total.set_value(index,'mkt_cap', mkt_cap)
    total.set_value(index,'kospi', kospi)
    total.set_value(index,'kosdaq', kosdaq)
    total.set_value(index,'trash', trash)       
    

In [71]:
total

,name,code,time,price,time_1,price_1,price_dif_1,sell_1,buy_1,volume_1,...,variation_3,post_num_3,unique_id_3,click_3,like_3,dislike_3,mkt_cap,kospi,kosdaq,trash
0,호텔신라,008770,2018-02-14 10:00,"83,400",2018-02-14 09:10,"84,700","1,800","84,800","84,700","29,336",...,"2,036",5,4,571,7,4,32419.0,1.0,0.0,0.0
1,대한광통신,010170,2018-02-14 10:00,"7,050",2018-02-14 09:10,"7,060",90,"7,060","7,050","101,713",...,"3,467",3,3,142,1,3,4176.0,0.0,1.0,0.0
2,포스코 ICT,022100,2018-02-14 10:00,"8,320",2018-02-14 09:10,"8,310",0,"8,330","8,310","44,311",...,"1,712",3,3,781,1,5,12953.0,0.0,1.0,0.0
3,한화케미칼,009830,2018-02-14 10:00,"32,350",2018-02-14 09:10,"32,700",550,"32,700","32,650","32,647",...,"3,102",6,6,706,3,8,55706.0,1.0,0.0,0.0
4,지엘팜텍,204840,2018-02-14 10:00,"4,760",2018-02-14 09:10,"4,680",50,"4,700","4,680","114,565",...,"2,844",4,4,970,8,12,1494.0,0.0,1.0,0.0
5,코웰패션,033290,2018-02-14 10:00,"5,350",2018-02-14 09:10,"5,270",90,"5,280","5,270","22,780",...,"1,530",1,1,121,1,0,4661.0,0.0,1.0,0.0
6,대한항공,003490,2018-02-14 10:00,"35,900",2018-02-14 09:10,"35,700",300,"35,750","35,700","7,560",...,"1,028",2,2,70,0,0,33338.0,1.0,0.0,0.0
7,대한뉴팜,054670,2018-02-14 10:00,"13,750",2018-02-14 09:10,"13,800",0,"13,800","13,750","2,556",...,365,1,1,64,3,3,2117.0,0.0,1.0,0.0
8,레드캡투어,038390,2018-02-14 10:00,"15,350",2018-02-14 09:10,"15,350",100,"15,350","15,200",7,...,1,0,0,0,0,0,1357.0,0.0,1.0,0.0
9,이마트,139480,2018-02-14 10:00,"290,000",2018-02-14 09:10,"283,500","5,000","284,000","283,500","8,469",...,75,1,1,23,2,0,86415.0,1.0,0.0,0.0
